We assume the attacker to be the previous owner. 
The previous owner, by obtaining the value of KT, can extract the values of r4 and r5 from the messages sent from the new owner to the tag and calculate the updated keys and tag's Pseudonym.

In [6]:
import random

# Function to perform bitwise XOR
def bitwise_xor(binary_str1, binary_str2):
    return ''.join(str(int(bit1) ^ int(bit2)) for bit1, bit2 in zip(binary_str1, binary_str2))

# Function to perform left rotation
def left_rotation(X, Y):
    wt_y = Y.count('1')  # Hamming weight of Y
    L = len(X)  # Calculate L based on the length of X
    wt_y_mod_L = wt_y % L
    return X[wt_y_mod_L:] + X[:wt_y_mod_L]

# Function to perform Right rotation
def right_rotation(X, Y):
    wt_y = Y.count('1')  # Hamming weight of Y
    L = len(X)  # Calculate L based on the length of X
    wt_y_mod_L = wt_y % L
    return X[-wt_y_mod_L:] + X[:-wt_y_mod_L]

# Function to generate a random binary string of a given length
def generate_random_binary_string(length):
    return ''.join(str(random.randint(0, 1)) for _ in range(length))

# Function to simulate the permutation function
def permutation(X, Y):
    assert len(X) == len(Y), "Bit strings X and Y must have equal length"
    m = Y.count('1')  # Hamming weight of Y
    km_indices = [i for i in range(len(Y)) if Y[i] == '1']
    kn_indices = [i for i in range(len(Y)) if Y[i] == '0']
    # Construct the permutation Z 
    Z = ''.join([X[k] for k in km_indices[::-1]]) + ''.join([X[k] for k in kn_indices[::-1]])
    return Z

def inverse_permutation(Z, Y):
    assert len(Z) == len(Y), "Bit strings Z and Y must have equal length"
    m = Y.count('1')  # Hamming weight of Y
    km_indices = [i for i in range(len(Y)) if Y[i] == '1']
    kn_indices = [i for i in range(len(Y)) if Y[i] == '0']
    # Split Z into km and kn parts
    Z_km = Z[:m][::-1]
    Z_kn = Z[m:][::-1]
    # Reconstruct X using Z, km_indices, and kn_indices
    X = ['0'] * len(Y)
    for i in range(m):
        X[km_indices[i]] = Z_km[i]
    for i in range(len(Y) - m):
        X[kn_indices[i]] = Z_kn[i]
    return ''.join(X)

In [7]:
# Initialization Phase
l = 128 # The length of the values
# Step 1: Generate a n-bit random value for IDSx
IDSx = generate_random_binary_string(l)

# Step 2: Generate a l-bit random value for IDn
IDn = generate_random_binary_string(l)

# Step 3: Generate a l-bit random value for KR (temporary shared secret)
KR = generate_random_binary_string(l)

# Step 4: Generate a l-bit random value for KT (temporary shared secret)
KT = generate_random_binary_string(l)

# Step 5: Use the permutation function to create index data
index_data = {"fx(IDS1, ID1)": permutation(IDSx, IDn)}

# Define l-bit binary values for K1, K2, and K3 
K1 = generate_random_binary_string(l)
K2 = generate_random_binary_string(l)
K3 = generate_random_binary_string(l)

# Define a l-bit binary value for IDo (Identifier of the old owner)
IDo = generate_random_binary_string(l)

# Print the generated values
print("Initialization Phase:")
print("Generated IDo:", IDo)
print("Generated K1:", K1)
print("Generated K2:", K2)
print("Generated K3:", K3)
print("Generated IDSx:", IDSx)
print("Generated IDn:", IDn)
print("Generated KR:", KR)
print("Generated KT:", KT)
print("Index Data:", index_data)

Initialization Phase:
Generated IDo: 01111000001001111100001010000001100100100110101001000010100100001100000001111010100100011100011000001011111110001011110101011001
Generated K1: 00111101011110011000100100001001111110110000001110100011000101010100001111110100010111001000100000101001100110110010110010000100
Generated K2: 00110001101101000111110001100100011010101001101101010001000101101000101010011010110101000100010100000010110101010010011000101101
Generated K3: 10010110101000110000100001000010110111110001011000011101100100111100011011000101111101100111001010000110111010100110101000100001
Generated IDSx: 01101000001100110000110101111111111001100101110001011111110010111010101111100000010101011111000000001000011100001010000011100011
Generated IDn: 00100010001101001111001100010111100110110000110010001010010001000100011110001110101010011000010010101011001100110111110010110010
Generated KR: 011010011100010011011001100011110101111110010010110001110101101000000100011100010011001001111110100011

In [8]:
# Main Protocol

# Step 1 & 2 (Based on Figure)
# RN

# Generate a random l-bit value for r1 using PRNG
r1 = generate_random_binary_string(l)

# Calculate A = r1 ⊕ KT
A = bitwise_xor(r1, KT)

# Send "hello" and A to the tag
message_to_tag = "hello"
#message_to_tag_binary = ''.join(format(ord(char), '08b') for char in message_to_tag)  # Convert "hello" to binary

# Concatenate the binary message and A
message_to_tag = (message_to_tag, A)

# Print the generated values and the message to the tag
#print("Generated r1:", r1)
#print("Generated A:", A)
print("Message to tag:", message_to_tag)

# Step 3 & 4
# Tag

# Extract r'1 from A by performing a bitwise XOR with KT
r1_prime = bitwise_xor(A, KT)
#print("r1_prime:", r1_prime)

# Calculate B' using the permutation function with inputs (r'1 ⊕ IDSx, KT)
r1_prime_xor_IDSx = bitwise_xor(r1_prime, IDSx)
B_prime = permutation(r1_prime_xor_IDSx, KT)

# Send IDSx and B' to the new owner (RN)
print("IDSx (to be sent to the new owner):", IDSx)
print("Generated B' (to be sent to the new owner):", B_prime)

# Step 5 & 6
# RN

# Calculate B using the permutation function with inputs (r1 ⊕ IDSx, KT)
r1_xor_IDSx = bitwise_xor(r1, IDSx)
B = permutation(r1_xor_IDSx, KT)

# Check if B equals B'
if B == B_prime:
    # Case 1: B equals B'
    # Generate a new random number r2 using PRNG
    r2 = generate_random_binary_string(l)
    #print("r2:", r2)
    
    # Calculate C = fx(r2 ⊕ IDSx, KR)
    r2_xor_IDSx = bitwise_xor(r2, IDSx)
    C = permutation(r2_xor_IDSx, KR)

    # Calculate D = Rot(r2, fx(IDn, KR))
    D = left_rotation(r2, permutation(IDn, KR))

    # Calculate fx(IDSx, IDn)
    fx_IDSx_IDn = permutation(IDSx, IDn)

    # Send IDSx, C, D, and fx(IDSx, IDn) to the old owner (RC)
    print("Generated IDSx (to be sent to the old owner):", IDSx)
    print("Generated C (to be sent to the old owner):", C)
    print("Generated D (to be sent to the old owner):", D)
    print("Generated fx(IDSx, IDn) (to be sent to the old owner):", fx_IDSx_IDn)

else:
    # Case 2: B is not equal to B', terminate the protocol
    print("B is not equal to B'. Protocol terminated.")


# Step 7 & 8
# RC

# Search for the corresponding index content in the index data table

if fx_IDSx_IDn in index_data.values():
    # Case 1: The result matches, ownership transfer request is legal

    # Extract r2 from C using fx, IDSx, and KR
    r2_xor_IDSx = inverse_permutation(C, KR)
    r2_prime = bitwise_xor(r2_xor_IDSx, IDSx)
    #print("r2_extracted:", r2_prime)
    
    # Recalculate D' = Rot(r'2, fx(IDn, KR))
    D_prime = left_rotation(r2_prime, permutation(IDn, KR))

    # Check if D' equals D
    if D_prime == D:
        # Calculate random number r3
        r3 = generate_random_binary_string(l)
        # Calculate E = fx(r3, r2) ⊕ KR
        E = bitwise_xor(permutation(bitwise_xor(r3, r2_prime), KR), KR)

        # Calculate F = Rot(r2 ⊕ KR, fx(IDo, r3))
        F = left_rotation(bitwise_xor(r2_prime, KR), permutation(IDo, r3))

        # Calculate G = fx(K1 ⊕ K2, K2 ⊕ K3)
        G = permutation(bitwise_xor(K1, K2), bitwise_xor(K2, K3))

        # Calculate H = Rot(K2 ⊕ K3, IDn ⊕ KR)
        H = left_rotation(bitwise_xor(K2, K3), bitwise_xor(IDn, KR))

        # Send E, F, G, and H to the new owner (RN)
        print("Generated E (to be sent to the new owner):", E)
        print("Generated F (to be sent to the new owner):", F)
        print("Generated G (to be sent to the new owner):", G)
        print("Generated H (to be sent to the new owner):", H)
    else:
        # D' is not equal to D, terminate the protocol
        print("D' is not equal to D. Protocol terminated.")
else:
    # Case 2: The result does not match, terminate the protocol
    print("Result does not match. Ownership transfer request is not legal. Protocol terminated.")


# Step 9 & 10
# RN

# Extract r'3 from E
E_xor_KR = bitwise_xor(E, KR)
r3_prime = inverse_permutation(E_xor_KR, r2)

# Calculate F' = Rot(r2 ⊕ KR, fx(IDo, r'3))
F_prime = left_rotation(bitwise_xor(r2, KR), permutation(IDo, r3_prime))

# Check if F' equals F
if F_prime == F:
    # Case 1: F is successfully verified
    #Extract K2 ⊕ K3 from H
    K2_xor_K3 = right_rotation(H, bitwise_xor(IDn, KR))
    
    # Generate a random number r4 using PRNG
    r4 = generate_random_binary_string(l)

    # Calculate I = fx(r4 ⊕ KT, K2 ⊕ K3)
    I = permutation(bitwise_xor(r4, KT), K2_xor_K3)

    # Calculate J = Rot(fx(r4, KT), r4 ⊕ IDSx)
    J = left_rotation(permutation(r4, KT), bitwise_xor(r4, IDSx))

    # Calculate G ⊕ I
    G_xor_I = bitwise_xor(G, I)
    
    # Send G ⊕ I and J to the tag
    print("Generated G ⊕ I (to be sent to the tag):", G_xor_I)
    print("Generated J (to be sent to the tag):", J)
else:
    # Case 2: F is not successfully verified, abort the protocol
    print("F is not successfully verified. Protocol aborted.")

    
# Step 11 & 12
# Tag

#Extract r'4 from G ⊕ I
K1_xor_K2_xor_r4_xor_KT = inverse_permutation(G_xor_I, bitwise_xor(K2, K3))
K1_xor_K2 = bitwise_xor(K1, K2)
K1_xor_K2_xor_KT = bitwise_xor(K1_xor_K2, KT)
r4_prime = bitwise_xor(K1_xor_K2_xor_r4_xor_KT, K1_xor_K2_xor_KT)

# Calculate J' = Rot(fx(r4, KT), r4 ⊕ IDSx)
r4_xor_IDSx = bitwise_xor(r4_prime, IDSx)
fx_r4_KT = permutation(r4_prime, KT)
J_prime = left_rotation(fx_r4_KT, r4_xor_IDSx)

# Check if J' == J
if J_prime == J:
    # Tag deems the old owner authorized to transfer ownership
    print("Tag deems the old owner authorized to transfer ownership")
    # Calculate L = fx(r4 ⊕ r1, KT) ⊕ r4
    r4_xor_r1 = bitwise_xor(r4, r1)
    L = bitwise_xor(permutation(r4_xor_r1, KT), r4)

    # Send L to the new owner
    print("L (to be sent to the new owner):", L)
else:
    # Tag does not authorize ownership transfer
    print("Ownership transfer not authorized.")
    

# Step 13 & 14
# RN

# Calculate L'
r4_xor_r1 = bitwise_xor(r4, r1)
L_prime = bitwise_xor(permutation(r4_xor_r1, KT), r4)

# Verify L and handle the cases
if L == L_prime:
    # Generate a random value for r5
    r5 = generate_random_binary_string(l)

    # Calculate M = fx(r4, KT) ⊕ r5
    fx_r4_KT = permutation(r4, KT)
    M = bitwise_xor(fx_r4_KT, r5)

    # Calculate N = Rot(r5, fx(r4, KT ⊕ IDSx))
    KT_xor_IDSx = bitwise_xor(KT, IDSx)
    fx_r4_KT_IDSx = permutation(r4, KT_xor_IDSx)
    N = left_rotation(r5, fx_r4_KT_IDSx)

    # Update pseudonym and shared keys
    IDSx_new = permutation(bitwise_xor(IDSx, r4), r5)
    #K1_xor_K2 = inverse_permutation(G, K2_xor_K3)
    K1_new = bitwise_xor(permutation(inverse_permutation(G, K2_xor_K3), r4), r5)
    K2_new = permutation(K2_xor_K3, r5)
    K3_new = permutation(bitwise_xor(KT, r4), r5)
 
    # Case 1: Send M, N to the tag. Update IDSx_new, K1_new, K2_new, K3_new, KT_new
    print("M (to be sent to the tag):", M)
    print("N (to be sent to the tag):", N)
    print("IDSx_new:", IDSx_new)
    print("K1_new:", K1_new)
    print("K2_new:", K2_new)
    print("K3_new:", K3_new)
    
else:
    # Case 2: abort the protocol
    print("Verification failed. Abort the protocol.")
    

# Step 15
# Tag

# Calculate r'5 from M
r5_prime = bitwise_xor(M, permutation(r4, KT))

# Calculate the local value of N
N_prime = left_rotation(r5_prime, permutation(r4, bitwise_xor(KT, IDSx)))

# Verify if the calculated N matches the received N
if N_prime == N:
    # Verification successful, update the pseudonym and shared secret
    IDSx_new = permutation(bitwise_xor(IDSx, r4), r5_prime)
    K1_new = bitwise_xor(permutation(bitwise_xor(K1, K2), r4), r5_prime)
    K2_new = permutation(bitwise_xor(K2, K3), r5_prime)
    K3_new = permutation(bitwise_xor(KT, r4), r5_prime)

    # Print the updated pseudonym and shared secret
    print("IDSx_new:", IDSx_new)
    print("K1_new:", K1_new)
    print("K2_new:", K2_new)
    print("K3_new:", K3_new)

else:
    # Verification failed
    print("Verification failed. Do not update pseudonym and shared secret.")

Message to tag: ('hello', '00011001011010001111111101111100001010011010100010101010001011000011001001011101110101111101110100001101110000011101011101001010')
IDSx (to be sent to the new owner): 01101000001100110000110101111111111001100101110001011111110010111010101111100000010101011111000000001000011100001010000011100011
Generated B' (to be sent to the new owner): 11000110111010101110000100000100010100001001111000001101110101101110110010110100101000010111000010111010110111101100000110111110
Generated IDSx (to be sent to the old owner): 01101000001100110000110101111111111001100101110001011111110010111010101111100000010101011111000000001000011100001010000011100011
Generated C (to be sent to the old owner): 01111010000100100010100111000001010010010001100101100111110110011001110100001100001100001011011001100110001000010110101010000100
Generated D (to be sent to the old owner): 110110110000101111011011101000010000010001011010100001100011110001101001011011011010111110111010000101100101001001

In [9]:
# Secret disclosure attack for obtaining KT:
# Attacker (Generate a random value to start the attack)
A1 = generate_random_binary_string(l)  # Send "hello" and A to the tag

#Tag (calculations)
r1_prime = bitwise_xor(A1, KT)
r1_prime_xor_IDSx = bitwise_xor(r1_prime, IDSx)
B_prime = permutation(r1_prime_xor_IDSx, KT)  # Send IDSx and B' to the attacker

#print("length:", l)
KT_R = [1] * l  # The initial value of KT for recovering

# Main loop for the attack
for i in range(l):
    
  # Attacker  
  e = [0 for b in range(l)]  
  e[i] = 1  # Standard basis
  e = "".join(str(element) for element in e) 
  A = bitwise_xor(A1, e)  # Sends A and "hello" to the Tag

  # Tag  
  r = bitwise_xor(A, KT)
  r_xor_IDSx = bitwise_xor(r, IDSx)
  B = permutation(r_xor_IDSx, KT)   # Sends IDSx and B' to the attacker

  # Attacker  
  delta = bitwise_xor(B_prime, B)   
  #print("delta:", delta)  
  m = KT_R[:i].count(0) 
  if delta[(l-1)-m] == '1':
    KT_R[i] = 0
  else:
    KT_R[i] = 1 

KT_R = "".join(str(bit) for bit in KT_R)
print("KT_R (Recovered):", KT_R)
print("KT == KT_R:", KT == KT_R)

KT_R (Recovered): 01001111101000110101001101011000111011110000110100110001000101101001100001111000100100011011011111010000001001110100000100010100
KT == KT_R: True


In [10]:
# Extraction of r4 and r5 is performed by tracking the messages G⊕I and M.

# Extract r'4 from G ⊕ I
K1_xor_K2_xor_r4_xor_KTR = inverse_permutation(G_xor_I, bitwise_xor(K2, K3))
K1_xor_K2 = bitwise_xor(K1, K2)
K1_xor_K2_xor_KTR = bitwise_xor(K1_xor_K2, KT_R)
r4_p = bitwise_xor(K1_xor_K2_xor_r4_xor_KTR, K1_xor_K2_xor_KTR)
print("r4_p (extracted):", r4_p)

# Calculate r'5 from M
r5_p = bitwise_xor(M, permutation(r4_p, KT_R))
print("r5_p (extracted):", r5_p)

# Obtaining the updated values by the previous owner
IDSx_n = permutation(bitwise_xor(IDSx, r4_p), r5_p)
K1_n = bitwise_xor(permutation(bitwise_xor(K1, K2), r4_p), r5_p)
K2_n = permutation(bitwise_xor(K2, K3), r5_p)
K3_n = permutation(bitwise_xor(KT_R, r4_p), r5_p)

# Checking the equality of values
print("IDSx_n:", IDSx_n)
print("K1_n:", K1_n)
print("K2_n:", K2_n)
print("K3_n:", K3_n)
print("Checking the equality of values:", IDSx_new==IDSx_n, K1_new==K1_n, K2_new==K2_n, K3_new==K3_n)

r4_p (extracted): 01010000001101000111110000100110010101001101101111001110110111001010100101110011111110100011010011011000011100111001111111100100
r5_p (extracted): 11000111010101101000100110001110001110010100010010110111000101111010101011011010100000101010111110000011101100111010100011101101
IDSx_n: 11000011011000001001001111010110001111100101100011001010011000000010111100000010101111010100000000000011000110011101001111000111
K1_n: 10010010010001011001100010011011110100010110001011011101011000000010111111000001111011001100111001001111001111010000010100100001
K2_n: 01100010011110001111010101111001001010001000101011110000011101110100001011101000010000010110010100011110100101010010011111000001
K3_n: 00011110100100000110001101100000100100111111111111111011011101110001011111010100000101011100101010111101010110010111110101001110
Checking the equality of values: True True True True
